Supermarkets around the world are using data mining techniques to analyze the user buying pattern in order to make their business more efficient thereby increasing their business and fulfilling customer needs at the same time. One such application is determining which are the goods that a consumer tend to buy together and analyze the 'if-then' patterns to understand if a consumer buys A which are the B's to recommend to the consumer. This helps them in placing the right products at the right aisle thereby helping the consumer recall their need. This helps the consumers to stock up their regrigerators and homes according to their needs and decreases their purchasing time at the same time.

In this project, I will use the Instacart's real dataset from Kaggle which contains data from 3 million grocery orders from 200,000 users. For each user, there are about 4-100 different orders based on how many times they have purchased from instacart.

I will be showing the implementation on spark owing to the fact that it runs very fast on databricks as it uses distributed in-process computing and takes very less time even on such a large dataset. Luckily we have the FP ( Frequent-Pattern Mining) library already on spark and thereby I will use that to understand the patterns. Let us start.

Note: Databricks provide users to use their community platform for free. So, feel free to replicate the code and see the magic of pattern mining

In [2]:
#importing necessary libraries
import pandas as pd
import numpy as np

In [3]:
#checking the files we have in the databricks file system

%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/__aisles-ca881.csv,__aisles-ca881.csv,226
dbfs:/FileStore/tables/__departments-c112e.csv,__departments-c112e.csv,226
dbfs:/FileStore/tables/__order_products__prior-5ba78.csv,__order_products__prior-5ba78.csv,226
dbfs:/FileStore/tables/__order_products__train-e8408.csv,__order_products__train-e8408.csv,226
dbfs:/FileStore/tables/__orders-074d8.csv,__orders-074d8.csv,226
dbfs:/FileStore/tables/__products-3d492.csv,__products-3d492.csv,226
dbfs:/FileStore/tables/aisles.csv,aisles.csv,2603
dbfs:/FileStore/tables/departments.csv,departments.csv,270
dbfs:/FileStore/tables/order_products__prior.csv,order_products__prior.csv,577550706
dbfs:/FileStore/tables/order_products__train.csv,order_products__train.csv,24680147


In [4]:
#enabling arrow just for a backup if pandas is needed at any point of time
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Generate a Pandas DataFrame
#pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a Pandas DataFrame using Arrow
#df = spark.createDataFrame(pdf)

# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
#result_pdf = df.select("*").toPandas()

/databricks/spark/python/pyspark/sql/session.py:577: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 arrow_schema = pa.Schema.from_pandas(pdf, preserve_index=False)

In [5]:
#Importing all the available files into the spark dataframe
aisles = spark.read.csv("/FileStore/tables/aisles.csv", header=True, inferSchema=True)
departments = spark.read.csv("/FileStore/tables/departments.csv", header=True, inferSchema=True)
order_products_prior = spark.read.csv("/FileStore/tables/order_products__prior.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv("/FileStore/tables/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv("/FileStore/tables/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("/FileStore/tables/products.csv", header=True, inferSchema=True)

# Create Temporary Tables to work using sql like commands
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

## Here's the data dictionary for all the files

## 1) orders (3.4m rows, 206k users):


order_id: order identifier

user_id: customer identifier

eval_set: which evaluation set this order belongs in (see SET described below)

order_number: the order sequence number for this user (1 = first, n = nth)

order_dow: the day of the week the order was placed on

order_hour_of_day: the hour of the day the order was placed on

days_since_prior: days since the last order, capped at 30 (with NAs for order_number = 1)




## 2) products (50k rows):


product_id: product identifier

product_name: name of the product

aisle_id: foreign key

department_id: foreign key




## 3) aisles (134 rows):


aisle_id: aisle identifier

aisle: the name of the aisle




## 4) departments (21 rows):


department_id: department identifier

department: the name of the department




## 5) order_products__SET (30m+ rows):


order_id: foreign key

product_id: foreign key

add_to_cart_order: order in which each product was added to cart


reordered: 1 if this product has been ordered by this user in the past, 0 otherwise
where SET is one of the four following evaluation sets (eval_set in orders):




"prior": orders prior to that users most recent order (~3.2m orders)

"train": training data supplied to participants (~131k orders)

"test": test data reserved for machine learning competitions (~75k orders)

Let us now explore what all data we have in the different files using the .show function in spark and do some basic exploratory data analysis to become familiar with the data.

In [8]:
#Top 5 orders in the orders dataframe
orders.show(n=5)

+--------+-------+--------+------------+---------+-----------------+----------------------+
order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
 2539329| 1| prior| 1| 2| 8| null|
 2398795| 1| prior| 2| 3| 7| 15.0|
 473747| 1| prior| 3| 3| 12| 21.0|
 2254736| 1| prior| 4| 4| 7| 29.0|
 431534| 1| prior| 5| 4| 15| 28.0|
+--------+-------+--------+------------+---------+-----------------+----------------------+
only showing top 5 rows

In [9]:
products.show(n=5)

+----------+--------------------+--------+-------------+
product_id| product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
 1|Chocolate Sandwic...| 61| 19|
 2| All-Seasons Salt| 104| 13|
 3|Robust Golden Uns...| 94| 7|
 4|Smart Ones Classi...| 38| 1|
 5|Green Chile Anyti...| 5| 13|
+----------+--------------------+--------+-------------+
only showing top 5 rows

In [10]:
order_products_train.show(n=5)

+--------+----------+-----------------+---------+
order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
 1| 49302| 1| 1|
 1| 11109| 2| 1|
 1| 10246| 3| 0|
 1| 49683| 4| 0|
 1| 43633| 5| 1|
+--------+----------+-----------------+---------+
only showing top 5 rows

In [11]:
order_products_prior.show(n=5)

+--------+----------+-----------------+---------+
order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
 2| 33120| 1| 1|
 2| 28985| 2| 1|
 2| 9327| 3| 0|
 2| 45918| 4| 1|
 2| 30035| 5| 0|
+--------+----------+-----------------+---------+
only showing top 5 rows

In [12]:
orders.show(n=5)

+--------+-------+--------+------------+---------+-----------------+----------------------+
order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
 2539329| 1| prior| 1| 2| 8| null|
 2398795| 1| prior| 2| 3| 7| 15.0|
 473747| 1| prior| 3| 3| 12| 21.0|
 2254736| 1| prior| 4| 4| 7| 29.0|
 431534| 1| prior| 5| 4| 15| 28.0|
+--------+-------+--------+------------+---------+-----------------+----------------------+
only showing top 5 rows

In [13]:
departments.show(n=5)

+-------------+----------+
department_id|department|
+-------------+----------+
 1| frozen|
 2| other|
 3| bakery|
 4| produce|
 5| alcohol|
+-------------+----------+
only showing top 5 rows

In [14]:
aisles.show(n=5)

+--------+--------------------+
aisle_id| aisle|
+--------+--------------------+
 1|prepared soups sa...|
 2| specialty cheeses|
 3| energy granola bars|
 4| instant foods|
 5|marinades meat pr...|
+--------+--------------------+
only showing top 5 rows

Let us now see how the total number of orders differ for different days of the week.

In [16]:
%sql
select 
  count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
   end) as day_of_week 
  from orders  
 group by order_dow 
 order by total_orders desc

total_orders,day_of_week
600905,Sunday
587478,Monday
467260,Tuesday
453368,Friday
448761,Saturday
436972,Wednesday
426339,Thursday


Most of the instacart orders are placed on Sunday or Monday. Seems like people recall their needs just after their weekend fun ends. Jokes apart, Let us also see the didstribution across the time of a day to analyze when does most of the users place an order from instacart.

In [18]:
%sql
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from orders 
 group by order_hour_of_day 
 order by order_hour_of_day

total_orders,hour
22758,0
12398,1
7539,2
5474,3
5527,4
9569,5
30529,6
91868,7
178201,8
257812,9


As we can see in the above bar plot, most of the orders are placed between 10 am and 4 pm. This was a bit surprising for me as I was expecting the peak to be outside office hours. Now, let us see which department is leading in terms of the number of products they have in the offerings.

In [20]:
%sql
  select department_id, count(1) as counter
    from products
   group by department_id
   order by counter desc 


department_id,counter
11,6563
19,6264
13,5371
7,4365
1,4007
16,3449
17,3084
15,2092
9,1858
4,1684


In [21]:
%sql
select
d.department_id,
d.department,
    count(1) as products
    from departments d
      inner join products p
         on p.department_id = d.department_id
   group by d.department_id, d.department 
   order by products desc

department_id,department,products
11,personal care,6563
19,snacks,6264
13,pantry,5371
7,beverages,4365
1,frozen,4007
16,dairy eggs,3449
17,household,3084
15,canned goods,2092
9,dry goods pasta,1858
4,produce,1684


In [22]:
%sql
select countbydept.*
  from (
  -- from product table, let's count number of records per dept
  -- and then sort it by count (highest to lowest) 
  select department_id, count(1) as counter
    from products
   group by department_id
   order by counter asc 
  ) as maxcount
inner join (
  -- let's repeat the exercise, but this time let's join
  -- products and departments tables to get a full list of dept and 
  -- prod count
  select
    d.department_id,
    d.department,
    count(1) as products
    from departments d
      inner join products p
         on p.department_id = d.department_id
   group by d.department_id, d.department 
   order by products desc
  ) countbydept 
  -- combine the two queries's results by matching the product count
  on countbydept.products = maxcount.counter

department_id,department,products
11,personal care,6563
19,snacks,6264
13,pantry,5371
7,beverages,4365
1,frozen,4007
16,dairy eggs,3449
17,household,3084
15,canned goods,2092
9,dry goods pasta,1858
4,produce,1684


Personal case followed by pantry and frozen leads in terms of the number of products they have. Now, let us visualize which products are the ones which are in the largest number of unique orders which means getting the most popular products.

In [24]:
%sql
select count(opp.order_id) as orders, p.product_name as popular_product
  from order_products_prior opp, products p
 where p.product_id = opp.product_id 
 group by popular_product 
 order by orders desc 
 limit 10

orders,popular_product
472565,Banana
379450,Bag of Organic Bananas
264683,Organic Strawberries
241921,Organic Baby Spinach
213584,Organic Hass Avocado
176815,Organic Avocado
152657,Large Lemon
142951,Strawberries
140627,Limes
137905,Organic Whole Milk


Bananas, Strawberries and Spinach leads in terms of the popularity. In fact, most of the top products seems healthy. When did America become so healthy?

In [26]:
%sql
select d.department, count(distinct p.product_id) as products
  from products p
    inner join departments d
      on d.department_id = p.department_id
 group by d.department
 order by products desc
 limit 10

department,products
personal care,6563
snacks,6264
pantry,5371
beverages,4365
frozen,4007
dairy eggs,3449
household,3084
canned goods,2092
dry goods pasta,1858
produce,1684


Next, we have to prepare our data in the form that can be fed into the pattern mining(FP growth) algorithm. We need each row to have a basket of items which were ordered together. Let us create a baskets dataframe for that before feeding into the algorithm.

In [28]:
# Organize the data by shopping basket
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')

In [29]:
rawData.show(5)

+--------------------+--------+
 product_name|order_id|
+--------------------+--------+
 Bulgarian Yogurt| 1|
Organic 4% Milk F...| 1|
Organic Celery He...| 1|
 Cucumber Kirby| 1|
Lightly Smoked Sa...| 1|
+--------------------+--------+
only showing top 5 rows

Here's how the baskets look like with each basket containing the set of  items which were purchased together.

In [31]:
baskets.show(5)

+--------+--------------------+
order_id| items|
+--------+--------------------+
 1342|[Raw Shrimp, Seed...|
 1591|[Cracked Wheat, S...|
 4519|[Beet Apple Carro...|
 4935| [Vodka]|
 6357|[Globe Eggplant, ...|
+--------+--------------------+
only showing top 5 rows

In [32]:
display(baskets)

order_id,items
1342,"List(Raw Shrimp, Seedless Cucumbers, Versatile Stain Remover, Organic Strawberries, Organic Mandarins, Chicken Apple Sausage, Pink Lady Apples, Bag of Organic Bananas)"
1591,"List(Cracked Wheat, Strawberry Rhubarb Yoghurt, Organic Bunny Fruit Snacks Berry Patch, Goodness Grapeness Organic Juice Drink, Honey Graham Snacks, Spinach, Granny Smith Apples, Oven Roasted Turkey Breast, Pure Vanilla Extract, Chewy 25% Low Sugar Chocolate Chip Granola, Banana, Original Turkey Burgers Smoke Flavor Added, Twisted Tropical Tango Organic Juice Drink, Navel Oranges, Lower Sugar Instant Oatmeal Variety, Ultra Thin Sliced Provolone Cheese, Natural Vanilla Ice Cream, Cinnamon Multigrain Cereal, Garlic, Goldfish Pretzel Baked Snack Crackers, Original Whole Grain Chips, Medium Scarlet Raspberries, Lemon Yogurt, Original Patties (100965) 12 Oz Breakfast, Nutty Bars, Strawberry Banana Smoothie, Green Machine Juice Smoothie, Coconut Dreams Cookies, Buttermilk Waffles, Uncured Genoa Salami, Organic Greek Whole Milk Blended Vanilla Bean Yogurt)"
4519,List(Beet Apple Carrot Lemon Ginger Organic Cold Pressed Juice Beverage)
4935,List(Vodka)
6357,"List(Globe Eggplant, Panko Bread Crumbs, Fresh Mozzarella Ball, Grated Parmesan, Gala Apples, Italian Pasta Sauce Basilico Tomato, Basil & Garlic, Organic Basil, Banana, Provolone)"
10362,"List(Organic Baby Spinach, Organic Spring Mix, Organic Leek, Slow Roasted Lightly Seasoned Chick'n, Organic Basil, Organic Shredded Mild Cheddar, Bag of Organic Bananas, Sliced Baby Bella Mushrooms, Organic Tapioca Flour, Organic Gala Apples, Lemons, Limes, Pitted Dates, Jalapeno Peppers, Original Tofurky Deli Slices, Organic Red Bell Pepper, Organic Shredded Carrots, Roma Tomato, Crinkle Cut French Fries, Large Greenhouse Tomato, Organic Pinto Beans, Organic Three Grain Tempeh, Organic Garnet Sweet Potato (Yam), Organic Coconut Milk, Organic Extra Firm Tofu, Ground Sausage Style Veggie Protein, Extra Virgin Olive Oil, Hass Avocados, Multigrain Tortilla Chips, The Ultimate Beefless Burger, Yellow Bell Pepper, Coconut Flour, Light Brown Sugar, Organic Harissa Seasoning, Crushed Garlic, Organic Whole Cashews)"
19204,"List(Reduced Fat Crackers, Dishwasher Cleaner, Peanut Powder, Disinfecting Wipes Lemon & Fresh Scent, Lemon Lime Thirst Quencher, Fat Free & Lower Sodium Chicken Broth, American Blend Salad, Cinnamon Cereal, Extra Nasal Strips, Reduced Fat Creamy Peanut Butter Spread, Mozzarella String Cheese Sticks, Light Low-Moisture Part Skim, Electrolyte Enhanced Water, Original Petroleum Jelly, High Efficiency Complete Dual Formula)"
29601,"List(Organic Red Onion, Small Batch Authentic Taqueria Tortilla Chips, Hummus, Hope, Original Recipe, Unsweetened Whole Milk Peach Greek Yogurt, Toasted Coconut Almondmilk Blend, Skillet Refried Red Beans Sautéed With Onion & Tomatillo, Almondmilk, Pure, Chocolate Protein, Organic Greek Lowfat Yogurt With Strawberries, Bag of Organic Bananas, California Orange Juice, Mini Whole Wheat Pita Bread, Coconut Almond Creamer Blend, Banana Chia Pod, Tomatillo Salsa, SALSA FRNTR CHPTL SALSA, Guacamole, Water)"
31035,"List(Organic Cripps Pink Apples, Organic Golden Delicious Apple, Organic Navel Orange, Bag of Organic Bananas)"
40011,"List(Organic Baby Spinach, Organic Blues Bread with Blue Cornmeal Crust, Sea Salt Macadamias, Natural Calm Magnesium Supplement Raspberry Lemon Flavor Powder, Chocolate Coconut Protein Bar, Sport Chocolate Mint Protein Bar)"


In [33]:
print((baskets.count(), len(baskets.columns)))

(131209, 2)

In total, we have about 1,31,209 basket of items. Now let us feed this data into the FPGrowth algorithm available in spark. Before doing that let us get some terms clear -

## 1) Support: 
This measure gives an idea of how frequent an itemset is in all the transactions. Intuitively, for any basket A, support measures the % of transactions containing that basket as a subset.

## 2) Confidence: 
This measure defines the likeliness of occurrence of consequent on the cart given that the cart already has the antecedents. Intuitively, let say there is a basket {a,b,c} having a support s, then if we are analyzing ({a} implies {b,c}), confidence is the % of the transactions having {a,b,c} that contains {b,c}

## 3) Lift: 
Lift controls for the support (frequency) of consequent while calculating the conditional probability of occurrence of {Y} given {X}. Lift is the most import parameter which supermarkets use to place products. Think of it as the *lift* that {X} provides to our confidence for having {Y} on the cart. To rephrase, lift is the rise in probability of having {Y} on the cart with the knowledge of {X} being present over the probability of having {Y} on the cart without any knowledge about presence of {X}.

Reference to understand these terms:

https://towardsdatascience.com/association-rules-2-aa9a77241654


Let us set the minimum support to 0.001, that means for our analysis, any basket that we will be analyzing should occur atleast 0.001* 1,31,209 ( 131) times to be considered in our frequent pattern analysis

In [35]:
%scala
import org.apache.spark.ml.fpm.FPGrowth

// Extract out the items 
val baskets_ds = spark.sql("select items from baskets").as[Array[String]].toDF("items")

// Use FPGrowth
val fpgrowth = new FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
val model = fpgrowth.fit(baskets_ds)

import org.apache.spark.ml.fpm.FPGrowth
baskets_ds: org.apache.spark.sql.DataFrame = [items: array<string>]
fpgrowth: org.apache.spark.ml.fpm.FPGrowth = fpgrowth_36d703db529d
model: org.apache.spark.ml.fpm.FPGrowthModel = fpgrowth_36d703db529d

In [36]:
%scala
// Display frequent itemsets
val mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")

mostPopularItemInABasket: org.apache.spark.sql.DataFrame = [items: array<string>, freq: bigint]

Now, let us see the most frequent basket of items.

In [38]:
%sql
select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20

items,freq
"List(Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas)",710
"List(Organic Raspberries, Organic Strawberries, Bag of Organic Bananas)",649
"List(Organic Baby Spinach, Organic Strawberries, Bag of Organic Bananas)",587
"List(Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas)",531
"List(Organic Hass Avocado, Organic Baby Spinach, Bag of Organic Bananas)",497
"List(Organic Avocado, Organic Baby Spinach, Banana)",484
"List(Organic Avocado, Large Lemon, Banana)",477
"List(Limes, Large Lemon, Banana)",452
"List(Organic Cucumber, Organic Strawberries, Bag of Organic Bananas)",424
"List(Limes, Organic Avocado, Large Lemon)",389


Now lets use the associationrules attribute of the fpgrowth algorithm to analyze the ifthen associations and see the confidence and lift values for different items. For a rule to be helpful to instacart, the lift value should be >1

In [40]:
%scala
// Display generated association rules.
val ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

ifThen: org.apache.spark.sql.DataFrame = [antecedent: array<string>, consequent: array<string> ... 2 more fields]

In [41]:
%sql
select * from ifThen where lift > 1 order by lift desc

antecedent,consequent,confidence,lift
List(Strawberry Rhubarb Yoghurt),List(Blueberry Yoghurt),0.3096646942800789,80.29801358062228
List(Blueberry Yoghurt),List(Strawberry Rhubarb Yoghurt),0.3102766798418972,80.29801358062227
List(Icelandic Style Skyr Blueberry Non-fat Yogurt),List(Nonfat Icelandic Style Strawberry Yogurt),0.2170212765957447,78.66062066533443
List(Nonfat Icelandic Style Strawberry Yogurt),List(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.42265193370165743,78.66062066533442
List(Icelandic Style Skyr Blueberry Non-fat Yogurt),List(Non Fat Acai & Mixed Berries Yogurt),0.2397163120567376,74.88794663964877
List(Non Fat Acai & Mixed Berries Yogurt),List(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.4023809523809524,74.88794663964876
List(Blackberry Cucumber Sparkling Water),List(Kiwi Sandia Sparkling Water),0.25675675675675674,72.44902644580064
List(Kiwi Sandia Sparkling Water),List(Blackberry Cucumber Sparkling Water),0.2860215053763441,72.44902644580063
List(Icelandic Style Skyr Blueberry Non-fat Yogurt),List(Non Fat Raspberry Yogurt),0.3120567375886525,71.08446611505121
List(Non Fat Raspberry Yogurt),List(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.3819444444444444,71.08446611505121


As we can see in the above table which has the rules in decreasing value of the lift values, if someone buys ["Strawberry Rhubarb Yoghurt"] there is a very high chance of buying ["Blueberry Yoghurt"]

Displaying in the order of confidence leads to the following. 

Note: Note quantifies the power of association that is uniquely because of the antecedent whereas confidence is just the probability of occurence of the consequent when there is an antecent.

In [43]:
%sql
select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence from ifThen order by confidence desc limit 20

antecedent (if),consequent (then),confidence
"List(Organic Raspberries, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.5984251968503937
"List(Organic Cucumber, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.546875
"List(Organic Kiwi, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5459770114942529
"List(Organic Navel Orange, Organic Raspberries)",List(Bag of Organic Bananas),0.5412186379928315
"List(Yellow Onions, Strawberries)",List(Banana),0.5357142857142857
"List(Organic Whole String Cheese, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5314685314685315
"List(Organic Navel Orange, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5283018867924528
"List(Organic Raspberries, Organic Hass Avocado)",List(Bag of Organic Bananas),0.521099116781158
"List(Organic D'Anjou Pears, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5170454545454546
"List(Organic Unsweetened Almond Milk, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5141065830721003
